## AMAZON REVIEW CLUSTERING - ksdeshpa

## Data Loading

In [9]:
import pandas as pd
import gzip
true = True

false = False
def parse(path):
    g=gzip.open(path, 'r')
    for l in g:
        yield eval(l)

def getDataFrame(path):
    i=0
    df ={}
    data = parse(path)
    for d in data:
        df[i] =d
        i += 1
        if i > 100000:
            break
    return pd.DataFrame.from_dict(df, orient ='index')

df = getDataFrame("C:/Users/Kedar/Desktop/AML/Amazon/Home_and_Kitchen.json.gz")

print(df.columns.values)

['overall' 'vote' 'verified' 'reviewTime' 'reviewerID' 'asin'
 'reviewerName' 'reviewText' 'summary' 'unixReviewTime' 'style' 'image']


In [10]:

corpus = df['reviewText'].values
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5.0,2,True,"08 31, 2010",A3NSN9WOX8470M,0006564224,mmm,"I don't use these for their original use, and ...",Many uses...,1283212800,NaN,NaN
1,5.0,2,True,"04 2, 2010",A2AMX0AJ2BUDNV,0006564224,John R. Welch,"Seems a bit expensive for a plastic bottle, bu...",Dispenser bottle,1270166400,NaN,NaN
2,5.0,NaN,True,"11 5, 2015",A8LUWTIPU9CZB,0560467893,Linda Fahner,"Great product, love it!!",Five Stars,1446681600,NaN,NaN
3,4.0,4,True,"10 29, 2015",AABKIIHAL0L66,0560467893,TheBlueChain,This is a sturdy floating corner shelf! We mo...,"Sturdy Shelf, Poor Installation Instructions",1446076800,NaN,NaN
4,3.0,NaN,True,"09 9, 2015",A3DA0KIQ5OBK5C,0560467893,angelaarden,I purchased 4 of these shelves. they look grea...,Look great - one bad one...,1441756800,NaN,NaN


## Data Preparation

In [33]:
import string,re
def clean_text(text): 

    # remove punctuation
    text_cleaned = "".join([x for x in text if x not in string.punctuation]) 
        
    # Remove extra white space 
    text_cleaned = re.sub(' +', ' ', text_cleaned)
    text_cleaned = text_cleaned.lower()
    
    tokens = text_cleaned.split(" ")  

    return text_cleaned
corpus = df['reviewText'].apply(lambda x:clean_text(x))

In [34]:
df.notnull().mean().sort_values()

image             0.008090
vote              0.178718
style             0.741553
summary           0.999870
reviewerName      0.999940
overall           1.000000
verified          1.000000
reviewTime        1.000000
reviewerID        1.000000
asin              1.000000
reviewText        1.000000
unixReviewTime    1.000000
dtype: float64

In [35]:
import nltk
from nltk.corpus import wordnet
# The WordNet Lemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()

# Acceptable parts of speech
pos_map = {'N': wordnet.NOUN, 'J': wordnet.ADJ, 'V': wordnet.VERB, 'R': wordnet.ADV }
# Function to reduce corpus to word lemmas, limited to nouns, verbs, etc. 
def clean_corpus(corpus):
    cleaned = []
    for text in corpus:
        words = []
        for sent in nltk.sent_tokenize(text):
            for (word, pos) in nltk.pos_tag(nltk.word_tokenize(sent)):
                if not pos[0] in pos_map:
                    continue
                word = lemmatizer.lemmatize(word.lower(), pos_map[pos[0]])
                words.append(word.lower())
        cleaned.append(words)
    return cleaned
clean_texts = clean_corpus(corpus)

In [37]:
from collections import Counter

# Print the 30 most frequent words, and add to stopword list
counter = Counter([w for t in clean_texts for w in t])
wordcounts = counter.most_common(n=30)
stopwords = []
for word, count in wordcounts:
    stopwords.append(word)
    print('%s%i' % (word.ljust(15), count))
    
stopwords.extend(['from','nice', 'like', 'Like',\
    'stand', 'great', 'well','good','product','put',\
    'buy', 'ago', 're', 'edu', 'use', 'from', 'my', 'we',\
    'i', 've', 'buy', 'set', 'lot', 'decide', 'give', 'add'])

be             179495
i              158518
have           75142
use            48124
not            33880
make           30360
great          28479
do             28027
work           24953
very           24748
good           24168
get            23710
so             22183
well           20364
buy            20281
just           19380
love           18238
time           18230
easy           18047
product        15532
up             14837
year           14504
more           12354
clean          11979
out            11322
only           11244
go             10339
much           10267
little         10183
need           10014


In [38]:
# Remove stopwords from each lemmatized text block
for i in range(len(clean_texts)):
    j = 0
    while j < len(clean_texts[i]):
        if clean_texts[i][j] in stopwords:
            del clean_texts[i][j]
        else:
            j += 1

In [39]:
# Show example text block after data preparation
print(clean_texts[0])

['dont', 'original', 'purchase', 'replacement', 'last', 'never', 'even', 'think', 'internet', 'find', 'new', 'one', 'pleasant', 'surprise', 'find', 'immediately', 'actually', 'pay', 'less']


## Creating corpus and model training

In [40]:
from gensim.corpora.dictionary import Dictionary

# Create a corpus from a list of texts
dictionary = Dictionary(clean_texts)
corpus = [dictionary.doc2bow(text) for text in clean_texts]

In [42]:
from gensim.models.ldamulticore import LdaMulticore

# Pretrain the model using the corpus and number of topics
num_topics = 20
lda = LdaMulticore(corpus, id2word=dictionary, num_topics=num_topics, passes=2)

## Get Coherence Metric

In [43]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=clean_texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4958895403220406


## Topic Inference

In [45]:
# Print the top 20 words in each learned topic
for i in range(num_topics):
    print('Topic %i: %s\n' % (i, lda.print_topic(i, topn=20)))

Topic 0: 0.153*"pan" + 0.053*"whisk" + 0.026*"nonstick" + 0.014*"spoon" + 0.012*"scratch" + 0.011*"toast" + 0.010*"ok" + 0.009*"stick" + 0.008*"perfectly" + 0.007*"silicone" + 0.007*"egg" + 0.006*"surface" + 0.006*"liner" + 0.006*"spray" + 0.006*"par" + 0.006*"look" + 0.005*"utensil" + 0.005*"inch" + 0.005*"other" + 0.005*"charm"

Topic 1: 0.032*"tea" + 0.029*"water" + 0.017*"hot" + 0.015*"kettle" + 0.012*"keep" + 0.008*"heat" + 0.008*"leave" + 0.008*"ive" + 0.008*"dont" + 0.008*"lid" + 0.008*"top" + 0.007*"still" + 0.006*"boil" + 0.006*"off" + 0.006*"stove" + 0.005*"cupcake" + 0.005*"dry" + 0.005*"doesnt" + 0.005*"take" + 0.005*"long"

Topic 2: 0.023*"blade" + 0.017*"handle" + 0.014*"cut" + 0.014*"bowl" + 0.011*"edge" + 0.011*"hand" + 0.010*"sharp" + 0.009*"knife" + 0.009*"hold" + 0.009*"too" + 0.008*"potato" + 0.008*"slice" + 0.008*"really" + 0.007*"sharpen" + 0.007*"small" + 0.007*"dont" + 0.007*"apple" + 0.006*"juice" + 0.006*"also" + 0.006*"kitchen"

Topic 3: 0.037*"pressure" + 0.

In [46]:
from gensim.test.utils import datapath
from gensim import models 
# Save the pretrained model to the datapath
dp = datapath("lda.model")
lda.save(dp)

# Load the pretrained model from the datapath
lda = models.ldamodel.LdaModel.load(dp)

## Exporting model using Pickle

In [47]:
import pickle
# save the model to disk
with open('C:/Users/Kedar/Desktop/AML/Amazon/amazon_clean.model','wb') as ama:
    pickle.dump(lda,ama)